In [ ]:
!pip install azure-storage-file-datalake 
!pip install azure-datalake-store
!pip install azure-identity

In [ ]:
from azure.datalake.store import core, lib
from azure.storage.filedatalake import DataLakeServiceClient
import os, uuid, sys
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core._match_conditions import MatchConditions
from azure.storage.filedatalake._models import ContentSettings
import pandas as pd
import pandas as pd
import io
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [ ]:
storage_account_name = ""
storage_account_key = ""

In [ ]:
def initialize_storage_account():  
    
    try:  
        global service_client

        service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format( 
            "https", storage_account_name), credential=storage_account_key)
    
    except Exception as e:
        print(e)

In [ ]:
def create_file_system():

  
  initialize_storage_account()

  global Container_name


  for retry in range(5):

    Container_name=str(input('Enter the Container name : '))

    file_system_client = service_client.get_file_system_client(file_system=Container_name)

    if file_system_client.exists()==False:

      try:

        file_system_client = service_client.create_file_system(file_system=Container_name)

        print("Container Successfully Created ")

        break

      except Exception as e:

        print(e)

        break

    else:

      print("Container Already exists")
      print("Enter different name ")

  else:
        
        print("you keep making invalid choices, exiting.")
        s= "you keep making invalid choices, exiting."
      
        return s
         

In [ ]:
def create_directory():

  initialize_storage_account()

  global directory_name

  for retry in range(5):

      directory_name=str(input('Enter the directory name : '))

      file_system_client = service_client.get_file_system_client(file_system=Container_name)


      if file_system_client.get_directory_client(directory_name).exists()==False:

        try:

          file_system_client.create_directory(directory_name)

          print('Directory created Successfully')

          break

        except Exception as e:

          print(e)

          break

      else:

        print("Directory Already exists")
        print("Enter different name ")


  else:
      print( "you keep making invalid choices, exiting.")

      s= "you keep making invalid choices, exiting."
      
      return s

In [ ]:
def upload_file():

  initialize_storage_account()

  
  wantto_createcontainer=str(input('Want to Create Container (Yes/No) : '))

  

  if wantto_createcontainer.capitalize()=='Yes':
    
    
    output=create_file_system()

    if output=="you keep making invalid choices, exiting.":

      return 'Execute the Function Again'


    wantto_createdirectory=str(input('Want to Create Directory (Yes/No) : '))

    if wantto_createdirectory.capitalize()=='Yes':

        out_put=create_directory() 

        if output=="you keep making invalid choices, exiting.":
          
          return 'Execute the Function Again'


        try:

          file_system_client = service_client.get_file_system_client(file_system=Container_name)  #need to mention the container name
          directory_client = file_system_client.get_directory_client(directory_name) #if we want the folder name
          file_client = directory_client.create_file(str(input('Enter the file name and extension to create : '))) #it will create an empty csv file
          local_file = open(str(input('Enter the Local path : ')),'r') #we are reading the csv file from local

          file_contents = local_file.read() #reading the file 

          file_client.append_data(data=file_contents, offset=0, length=len(file_contents)) #addeding the data to empty csv file

          file_client.flush_data(len(file_contents)) # commiting the append of data

          print("File uploaded file to directory")

        except Exception as e:
          print(e)

    else:
        try:

          file_system_client = service_client.get_file_system_client(file_system=Container_name)  #need to mention the container name
          directory_client = file_system_client.get_directory_client(str(input('Enter the directory name : '))) #if we want the folder name
          file_client = directory_client.create_file(str(input('Enter the file name and extension to create : '))) #it will create an empty csv file
          local_file = open(str(input('Enter the Local path : ')),'r') #we are reading the csv file from local
          file_contents = local_file.read() #reading the file 

          file_client.append_data(data=file_contents, offset=0, length=len(file_contents)) #addeding the data to empty csv file

          file_client.flush_data(len(file_contents)) # commiting the append of data
          print("File uploaded file to directory")

        except Exception as e:
          print(e)

  else:

    try:

      file_system_client = service_client.get_file_system_client(file_system=str(input('Enter the Container name : ')))  #need to mention the container name

      file_client=file_system_client.create_file(str(input('Enter the file name and extension to create : ')))
      local_file = open(str(input('Enter the Local path : ')),'r') #we are reading the csv file from local

      file_contents = local_file.read() #reading the file 

      file_client.append_data(data=file_contents, offset=0, length=len(file_contents)) #addeding the data to empty csv file

      file_client.flush_data(len(file_contents)) # commiting the append of data
      print("File uploaded")

    except Exception as e:
        print(e)




In [ ]:
upload_file()

In [ ]:
def read_file():

  container_name =str(input('Enter the Container name : '))

  file_system_client = service_client.get_file_system_client(file_system=container_name)

  wantto_readfromdirectory=str(input('Want to read from Directory (Yes/No) : '))

  if wantto_readfromdirectory.capitalize()=='Yes':


    directory_name=str(input('Enter the directory name : '))

    directory_client = file_system_client.get_directory_client(directory_name)

    file_client = file_system_client.get_file_client(str(input('Enter the file path : ')))

    download = file_client.download_file()

    downloaded_bytes = download.readall()

    data_string=downloaded_bytes.decode('UTF-8')
    data = io.StringIO(data_string)
    df = pd. read_csv(data)

    return df

  else:

    file_client = file_system_client.get_file_client(str(input('Enter the Filename with extension : ')))

    download = file_client.download_file()

    downloaded_bytes = download.readall()

    data_string=downloaded_bytes.decode('UTF-8')
    data = io.StringIO(data_string)
    df = pd. read_csv(data)

    return df

In [ ]:
df=read_file()

In [ ]:
df